<a href="https://colab.research.google.com/github/rwsiegel/DS5230_UML_Project/blob/main/document-reranking_rs_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Reranking



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a reranking function that takes the top 1000 results of the baseline model and tries to make more relevant documents rank higher.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library was written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, we make sure we point to an appropriate Java installation. If you don't have Java 11, this would need to be changed.

In [1]:
%%capture
!pip install pyserini==0.12.0

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

You can use the `SimpleSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already:

In [2]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('robust04')

Attempting to initialize pre-built index robust04.


index-robust04-20191213.tar.gz: 1.70GB [01:33, 19.6MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-robust04-20191213.tar.gz into /root/.cache/pyserini/indexes/index-robust04-20191213.15f3d001489c97849a010b0a4734d018...
Initializing robust04...


Now we can search for a query and inspect the results:

In [3]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `hits` object also contains the raw text of the documents in the index before processing. In other words, this version of the text has not been divided into fields, tokens, etc.

In [4]:
hits[0].raw

'<DATE>\n<P>\nSeptember 27, 1990, Thursday, Ventura County Edition\n</P>\n</DATE>\n<HEADLINE>\n<P>\nHUNGRY WILDLIFE STRAYING INTO SUBURBS;\n</P>\n<P>\nDROUGHT: FOUR DRY YEARS HAVE PARCHED NATIVE VEGETATION, FORCING BOBCATS, BEARS,\nMOUNTAIN LIONS, DEER AND COYOTES TO FORAGE CLOSER TO INHABITED AREAS.\n</P>\n</HEADLINE>\n<TEXT>\n<P>\nHungry bobcats, bears and mountain lions -- unable to find food in Ventura\nCounty\'s drought-parched forests -- are being pushed out of their natural\nhabitats to scavenge in rural communities, game officials said Wednesday.\n</P>\n<P>\nTwo weeks ago, a black bear ripped the door off a trailer home in Rose Valley\njust north of Ojai. And within the past month, there have been several reports\nof mountain lions eating livestock near Los Padres National Forest. Several\nbobcats have been reported near houses in the Ojai Valley.\n</P>\n<P>\nAuthorities say that over the past two years they have received twice the\ncomplaints -- about 20 a month -- of wild ani

The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [5]:
from pyserini.index import IndexReader
from IPython.core.display import display, HTML

reader = IndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Attempting to initialize pre-built index robust04.
/root/.cache/pyserini/indexes/index-robust04-20191213.15f3d001489c97849a010b0a4734d018 already exists, skipping download.
Initializing robust04...


Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [6]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [7]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

{'advis': 2,
 'anim': 9,
 'area': 4,
 'attack': 3,
 'author': 3,
 'bear': 4,
 'been': 11,
 'black': 2,
 'bobcat': 4,
 'california': 2,
 'cat': 3,
 'counti': 4,
 'coyot': 10,
 'debusscher': 3,
 'deer': 3,
 'depart': 2,
 'drought': 4,
 'dry': 2,
 'elsewher': 2,
 'especi': 2,
 'famili': 2,
 'few': 2,
 'food': 3,
 'forest': 2,
 'game': 2,
 'ha': 3,
 'habitat': 2,
 'have': 16,
 'he': 2,
 'hi': 2,
 'hous': 3,
 'hungri': 3,
 'i': 2,
 'jenk': 4,
 'just': 3,
 'leav': 2,
 'lion': 3,
 'live': 2,
 'lot': 2,
 'month': 4,
 'more': 3,
 'mountain': 3,
 'natur': 2,
 'near': 3,
 'off': 2,
 'offici': 5,
 'ojai': 3,
 'on': 2,
 'out': 4,
 'parch': 2,
 'park': 2,
 'past': 2,
 'peopl': 2,
 'rees': 3,
 'report': 4,
 'resid': 2,
 'rural': 3,
 'sai': 2,
 'said': 19,
 'seen': 3,
 'septemb': 2,
 'sever': 3,
 'she': 3,
 "they'r": 5,
 'two': 3,
 'vallei': 4,
 'ventura': 4,
 'we': 2,
 'who': 2,
 'wild': 3,
 'wildlif': 2,
 'would': 2,
 'yard': 2,
 'year': 3}

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the Trec conferences call them.

In [8]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: Trec-speak for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [9]:
topics[301]

{'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [10]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

Now, we'll fetch the relevance judgments for the Robust04 queries, which Trec calls "qrels".

In [11]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
#qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [12]:
qrels[0:10]

[[301, 0, 'FBIS3-10082', 1],
 [301, 0, 'FBIS3-10169', 0],
 [301, 0, 'FBIS3-10243', 1],
 [301, 0, 'FBIS3-10319', 0],
 [301, 0, 'FBIS3-10397', 1],
 [301, 0, 'FBIS3-10491', 1],
 [301, 0, 'FBIS3-10555', 0],
 [301, 0, 'FBIS3-10622', 1],
 [301, 0, 'FBIS3-10634', 0],
 [301, 0, 'FBIS3-10635', 0]]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

In [13]:
hits = searcher.search(dev_topics[355]['title'], 1000)
[(hit.docid, hit.score) for hit in hits[0:10]]

[('FBIS4-20436', 11.349300384521484),
 ('FBIS3-23683', 9.126500129699707),
 ('FBIS3-21238', 8.309300422668457),
 ('FBIS4-44915', 7.935699939727783),
 ('FBIS4-20602', 7.6006999015808105),
 ('FBIS4-47382', 7.529600143432617),
 ('FT943-1589', 7.480100154876709),
 ('LA071789-0059', 7.451399803161621),
 ('FBIS4-22145', 7.215099811553955),
 ('FBIS4-44667', 7.106500148773193)]

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [14]:
# need to add in the relevance judgement to the query, the search will turn up documents in a list in an order
# of the scoring metric. So once the scoring metrics are delivered, 'join' in the corresponding relevance of the given document and the given query

In [15]:
## TODO: Compute MAP@1000 for test_topics
dev_topics[350]

{'description': 'Is it hazardous to the health of individuals to\twork with computer terminals on a daily basis?',
 'narrative': 'Relevant documents would contain any information that  expands on any physical disorder/problems that may be associated with the daily working with computer terminals. Such things as carpel tunnel, cataracts, and fatigue have  been said to be associated, but how widespread are these or other problems and what is being done to alleviate any health problems.',
 'title': 'Health and Computer Terminals'}

In [43]:
import pdb
for topic in dev_topics:
  pdb.set_trace()
  hits = searcher.search(topics[topic]['title'], 1000)
  [(hit.docid, hit.score) for hit in hits[0:10]]

  


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-43-e682e3bc4d84>(4)<module>()
-> hits = searcher.search(topics[topic]['title'], 1000)
(Pdb) topic
350
(Pdb) topics[topic]
{'narrative': 'Relevant documents would contain any information that  expands on any physical disorder/problems that may be associated with the daily working with computer terminals. Such things as carpel tunnel, cataracts, and fatigue have  been said to be associated, but how widespread are these or other problems and what is being done to alleviate any health problems.', 'description': 'Is it hazardous to the health of individuals to\twork with computer terminals on a daily basis?', 'title': 'Health and Computer Terminals'}
(Pdb) topics[topic]['title']
'Health and Computer Terminals'
(Pdb) exit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

In [51]:
def get_map_1000(topics, rel_queries):
  MAP_results = []
  df_rel_queries = pd.DataFrame(rel_queries, columns = ['document_id', 'round', 'document', 'relevance'])
  
  for topic in topics:
    pdb.set_trace()
    hits = searcher.search(topics[topic]['title'], 1000)
    query_results = [(hit.docid, hit.score) for hit in hits[0:1000]]

    df_rel_queries_topic = df_rel_queries[df_rel_queries['document_id'] == topic]

    df_query_results = pd.DataFrame(query_results, columns = ['document', 'score'])

    all_df = pd.merge(df_query_results, df_rel_queries_topic, how = 'inner', left_on = 'document', right_on = 'document')

    all_df.reset_index(inplace = True)
    all_df['rank'] = all_df['index']+1
    all_df.drop(columns = 'index') 

    all_df['precision'] = (all_df['relevance'].cumsum(axis = 0))/all_df['rank'])


In [52]:
get_map_1000(dev_topics, qrels)

> <ipython-input-51-2e540667c425>(7)get_map_1000()
-> hits = searcher.search(topics[topic]['title'], 1000)
(Pdb) topic
350
(Pdb) topics[topic]['title']
'Health and Computer Terminals'
(Pdb) hits = searcher.search(topics[topic]['title'], 1000)
(Pdb) query_results = [(hit.docid, hit.score) for hit in hits[0:1000]]
(Pdb) df_rel_queries_topic = df_rel_queries[df_rel_queries['document_id'] == topic]
(Pdb) df_rel_queries_topic.head()
       document_id  round     document  relevance
60226          350      0  FBIS3-10261          0
60227          350      0  FBIS3-10291          0
60228          350      0  FBIS3-10855          0
60229          350      0  FBIS3-10875          0
60230          350      0  FBIS3-10963          0
(Pdb) df_query_results = pd.DataFrame(query_results, columns = ['document', 'score'])
(Pdb) all_df = pd.merge(df_query_results, df_rel_queries_topic, how = 'inner', left_on = 'document', right_on = 'document')
(Pdb) all_df.head()
        document   score  document_id 

BdbQuit: ignored

In [50]:
df_rel_queries = pd.DataFrame(qrels, columns = ['document_id', 'round', 'document', 'relevance'])
df_rel_queries.head()

,document_id,round,document,relevance
0,301,0,FBIS3-10082,1
1,301,0,FBIS3-10169,0
2,301,0,FBIS3-10243,1
3,301,0,FBIS3-10319,0
4,301,0,FBIS3-10397,1


In [16]:
import pdb
health_and_comp_terminals = []
for v in qrels:
  #pdb.set_trace()
  if v[0] == 350:
    #pdb.set_trace()
    health_and_comp_terminals.append(v)

In [17]:
len(health_and_comp_terminals)

1285

In [18]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


In [19]:
hits = searcher.search(dev_topics[350]['title'], 1000)
health_and_comp_results = [(hit.docid, hit.score) for hit in hits[0:10]]

In [35]:
import pandas as pd
health_and_comp_df = pd.DataFrame(health_and_comp_terminals, columns = ['document_id', 'round', 'document', 'relevance'])

test_df = pd.DataFrame(health_and_comp_results, columns = ['document', 'score'])

test_all_df = pd.merge(test_df, health_and_comp_df, how = 'inner', left_on = 'document', right_on = 'document')

In [36]:
test_all_df.reset_index(inplace = True)
test_all_df['rank'] = test_all_df['index']+1
test_all_df.drop(columns = 'index')


,document,score,document_id,round,relevance,rank
0,LA052290-0188,8.4590,350,0,1,1
1,LA060690-0112,7.7001,350,0,1,2
2,FT922-6787,7.6209,350,0,0,3
3,FT931-7146,7.4038,350,0,0,4
4,FT923-4291,7.0596,350,0,1,5
5,FR940805-2-00111,6.9839,350,0,0,6
6,LA100690-0107,6.8678,350,0,1,7
7,FR940822-0-00016,6.8654,350,0,0,8


In [42]:
test_all_df['precision'] = (test_all_df['relevance'].cumsum(axis = 0))/test_all_df['rank'])

TypeError: ignored

In [41]:
test_all_df

,index,document,score,document_id,round,relevance,rank,precision
0,0,LA052290-0188,8.4590,350,0,1,1,1.000000
1,1,LA060690-0112,7.7001,350,0,1,2,1.000000
2,2,FT922-6787,7.6209,350,0,0,3,0.666667
3,3,FT931-7146,7.4038,350,0,0,4,0.500000
4,4,FT923-4291,7.0596,350,0,1,5,0.600000
5,5,FR940805-2-00111,6.9839,350,0,0,6,0.500000
6,6,LA100690-0107,6.8678,350,0,1,7,0.571429
7,7,FR940822-0-00016,6.8654,350,0,0,8,0.500000


In [32]:
health_and_comp_terminals[0:10]

[[350, 0, 'FBIS3-10261', 0],
 [350, 0, 'FBIS3-10291', 0],
 [350, 0, 'FBIS3-10855', 0],
 [350, 0, 'FBIS3-10875', 0],
 [350, 0, 'FBIS3-10963', 0],
 [350, 0, 'FBIS3-11090', 0],
 [350, 0, 'FBIS3-11984', 0],
 [350, 0, 'FBIS3-14832', 0],
 [350, 0, 'FBIS3-1590', 0],
 [350, 0, 'FBIS3-17040', 0]]

In [39]:
import pandas as pd
health_and_comp_df = pd.DataFrame(health_and_comp_terminals)

In [42]:
health_and_comp_df[health_and_comp_df[2]=='LA052290-0188']

,0,1,2,3
1085,350,0,LA052290-0188,1


In [ ]:
hits[0]

## Reranking Search Results

The default `SimpleSearcher` in pyserini uses a BM25 model. In this final part of the assignment, you should implement a different, and hopefully improved ranking function. To make this easier to implement, you will _re_-rank the top 1000 results for each query that you evaluated above. In other words, rather than retrieving documents from the whole collection, scan through the top 1000 results for each query given by the baseline SimpleSearcher BM25 model and compute a new score for that result. Then re-sort the top-1000 results by your model's score.

You may use anything you've learning in this course—or in another course—to build your ranking function. For example, you could implement pseudo-relevance feedback or a relevance model, which would treat the top of each ranked list (e.g., the top 100) as if it were truly relevant and retrain model parameters. You could tune different BM25, query likelihood, or sequential dependence models. You could try to learn different weights or embeddings for different fields in documents. You could use implementations of transformer language models such as [BERT](https://github.com/castorini/anserini-notebooks/blob/master/Pyserini+SciBERT_on_COVID_19_Demo.ipynb) or [SentenceBERT](https://www.sbert.net/examples/training/ms_marco/README.html) to score the compatibility of queries and documents. To be clear, you  don't have to try all of these approaches, nor do you have to try any of them. You are free to try whatever ideas you like.

If your reranking model has tunable parameters, you should tune them on the `dev_topics` set. In any case, you should evaluate its MAP@1000 on the `test_topics` set.

In [ ]:
## TODO: Implement a raranking function that takes a query and a list of results and computes a new score.
## Lile the original score of the BM25 baseline model, a higher score should mean a better result.

In [ ]:
## TODO: Evaluate your reranker's MAP@1000 on test_topics.